In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

## Train-Test Influencers:

In [2]:
cascades_file = "./data/total.txt"
f = open(cascades_file, "r+") 
ids = set()
counter = 0
train_influencers = {}
test_influencers = {}
n=5
    
for line in f: 
        
    date = line.split(" ")[1].split("-")
    original_user_id = line.split(" ")[2]
        
    retweets = next(f).replace(" \n","").split(" ") #****************************CHANDU: Updated*******************************************
        #----- keep only the cascades and the nodes that are active in train (2011.10.29 -2012.9.28) and test (2012.9.28 -2012.10.29)
           
    retweet_ids = []        
        #------- last month at test
    if int(date[0])==2012 and ((int(date[1])>=9 and int(date[2])>=28)  or (int(date[1])==10  and int(date[2])<=29)):
        ids.add(original_user_id)
        for i in range(0,len(retweets)-1,2):
            ids.add(retweets[i])
            retweet_ids.append(retweets[i])
        if original_user_id not in test_influencers:
            test_influencers[original_user_id] =  retweet_ids
        else:
            test_influencers[original_user_id] =  test_influencers[original_user_id] + retweet_ids
            

    
       #------ The rest are used for training
    elif (int(date[0])==2012 and int(date[1])<9 and int(date[2])<28) or (int(date[0])==2011 and int(date[1])>=10 and int(date[2])>=29):
        ids.add(original_user_id)          
        for i in range(0,len(retweets)-1,2):
            ids.add(retweets[i])
            retweet_ids.append(retweets[i])
        if original_user_id not in train_influencers:
            train_influencers[original_user_id] =  retweet_ids
        else:
            train_influencers[original_user_id] =  train_influencers[original_user_id] + retweet_ids
            
            
           
    counter+=1    
    if (counter % 50000==0):
        print("------------"+str(counter))
f.close()
    

------------50000
------------100000
------------150000
------------200000


In [3]:
len(train_influencers)

26158

In [4]:
len(test_influencers)

8641

In [5]:
train_inflencers_list = [ key for key in train_influencers.keys()]
train_inflencers_set = set(train_inflencers_list)
train_inflencers_list[:5]

['1500872', '153602', '519514', '490872', '218165']

In [6]:
len(train_inflencers_list)

26158

In [7]:
test_inflencers_list = [ key for key in test_influencers.keys()]
test_inflencers_set = set(test_inflencers_list)
test_inflencers_list[:5]

['280058', '1674506', '519514', '497197', '1297718']

In [8]:
len(test_inflencers_list)

8641

In [9]:
train_test_inflencers_set = train_inflencers_set.intersection(test_inflencers_set)
len(train_test_inflencers_set)

4940

In [10]:
list(train_test_inflencers_set)[:5]

['799852', '475590', '1419120', '151197', '833664']

In [11]:
infected_length_dict = {}

In [12]:
for e in list(train_test_inflencers_set):
    infected_length_dict[e] = len(train_influencers[e]) + len(test_influencers[e])
    

In [17]:
infected_length_dict= {k: v for k, v in sorted(infected_length_dict.items(), key=lambda item: -item[1])}

In [18]:
biased_influencers = [ key for key in infected_length_dict.keys()][:n]
biased_influencers

['519514', '490872', '482551', '820273', '235238']

##  Create Gender Dictionary:

In [19]:
gender_dict = {}
for index in biased_influencers:
    for el in train_influencers[index]:
        if el not in gender_dict:
            gender_dict[el] = '1'
     
    for el in test_influencers[index]:
        if el not in gender_dict:
            gender_dict[el] = '1'
            

In [20]:
len(gender_dict)

562575

In [21]:
for user_id in list(ids):
    if user_id not in gender_dict:
        gender_dict[user_id] = str(np.random.choice([0,1], 1 ,replace=True, p=[0.65,0.35])[0])

In [22]:
len(gender_dict)

1170873

In [23]:
len(ids)

1170873

## Calculate Male Fraction:

In [24]:
male_list = []
female_list = []
for key in gender_dict.keys():
    if gender_dict[key] == '1':
        male_list.append(1)
    else:
        female_list.append(1)
        

In [25]:
len(male_list)

775142

In [26]:
len(female_list)

395731

In [27]:
len(male_list) + len(female_list)

1170873

In [28]:
len(male_list) / (len(male_list) + len(female_list))

0.6620205607269106

In [29]:
ids = list()
for i in gender_dict.keys():
    ids.append(i)
    
ids[:10]    

['537924',
 '169868',
 '1490395',
 '1336687',
 '1205682',
 '1715509',
 '868627',
 '377807',
 '175124',
 '777557']

In [30]:
ids_gender = list()
for i in gender_dict.values():
    ids_gender.append(i)
    
ids_gender[:10] 

['1', '1', '1', '1', '1', '1', '1', '1', '1', '1']

In [31]:
df_profile =pd.DataFrame({   "user_id":ids,"gender":ids_gender})

In [32]:
df_profile

,user_id,gender
0,537924,1
1,169868,1
2,1490395,1
3,1336687,1
4,1205682,1
...,...,...
1170868,1124789,0
1170869,1426408,0
1170870,1688856,0
1170871,278798,0


In [33]:
df_profile.to_csv("./data/profile_gender.csv")

## Fraction of Males Plot:

In [169]:
path = "./data/total.txt"
with open(path, 'r',  encoding="ISO-8859-1") as f:
    contents = f.read()
split_content = contents.split('\n')
split_content = split_content[0:-1]
split_content_list=[]
for i in split_content:
    split_content_list.append(i.split())

KeyboardInterrupt: 

In [ ]:
print(split_content_list[0])
print(split_content_list[1])

In [ ]:
odd_rows = []
for index,el in enumerate(split_content_list):
    if index%2 ==0:
        el=el
        odd_rows.append(el)

In [ ]:
even_rows = []
for index,el in enumerate(split_content_list):
    if index%2 ==1:
             even_rows.append(el)

In [ ]:
odd_rows[0], even_rows[0]

In [ ]:
y = even_rows[:2]
print(y)

In [ ]:
cascades_male_list = []
for C in even_rows:
    cascades_gender = []
    for index in range(len(C)):        
        if index%2 == 0:
            cascades_gender.append(gender_dict[C[index]])
    cascades_male_list.append(cascades_gender)
    


In [ ]:
gender_dict['864182']

In [ ]:
len(cascades_male_list)

In [ ]:
cascades_male_list

In [ ]:
cascades_male_array = np.array(cascades_male_list)

In [ ]:
plt.figure(figsize=(20,10))
plt.style.use('ggplot')
plt.hist(cascades_male_array, bins = 50, color = 'teal', alpha = 0.5)
plt.xlabel('fraction of males per cascade', fontsize=18, fontweight='bold')
plt.xticks(rotation=90)
plt.title("Distribution of Males Fraction in Test Cascades",  fontsize=22)
plt.tick_params(axis='both', which='major', labelsize=16)
plt.show()
plt.savefig("males_fraction_test_cascades.png")